# Prediction for RoBERTa after hyperparameter tuning




After experimenting with several parameter combinations that seemed plausible giving the hyperparameter tuning runs, we found
a learning rate = 0.00002 and number of epochs = 13 to fine tune RoBERTa to beat the baseline.


# Code

##Colab Phython Setup

###Connect to google drive and set the correct working directory


In [27]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [28]:
import os
os.chdir('/content/gdrive/MyDrive/TxtClassComp_MattzeePrivate/')

In [29]:
!pwd 

/content/gdrive/MyDrive/TxtClassComp_MattzeePrivate


### Install required libraries/frameworks

In [30]:
!pip install jsonlines
!pip install pandas

In [31]:
!pip install simpletransformers

### Load libraries

In [32]:
from simpletransformers.classification import ClassificationModel,ClassificationArgs
import pandas as pd
import numpy as np
import logging
import json
import sklearn
from statistics import mean, mode
import os
#import json
import jsonlines
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_recall_fscore_support


## Load and prepare the data

In [33]:
test_file = '/content/gdrive/MyDrive/TxtClassComp_MattzeePrivate/data/test.jsonl'
train_file = '/content/gdrive/MyDrive/TxtClassComp_MattzeePrivate/data/train.jsonl'

data_train = []
iter = 1
with jsonlines.open(train_file) as f:
    for line in f.iter():
        #data = json.load(line)
        #print(line) # or whatever else you'd like to do
        #print('processing training line: ' + str(iter))
        iter +=1
        data_train.append(line)
        #data = json.loads(line)
        #print(data)


data_test = []
iter = 1
with jsonlines.open(test_file) as f:
    for line in f.iter():
        #data = json.load(line)
        #print(line) # or whatever else you'd like to do
        #print('processing test line: ' + str(iter))
        iter +=1
        data_test.append(line)
        #data = json.loads(line)
        #print(data)
print("Count of training data entries:")
print(len(data_train))
print("Count of test data entries:")
print(len(data_test))

Count of training data entries:
5000
Count of test data entries:
1800


In [34]:
train_data_pd = pd.DataFrame.from_dict(data_train)
test_data_pd = pd.DataFrame.from_dict(data_test)
print("Training and Test Datasets converted to Pandas DataFrames...")

Training and Test Datasets converted to Pandas DataFrames...


In [35]:
#!pip install nltk
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, SpaceTokenizer
from nltk.stem import PorterStemmer

#print(stopwords.words('english'))

#for j in stopwords.words('english'):
#    print(j)

#print(stopwords.words())




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [36]:

#print(train_data_pd['context'][1])
#print(train_data_pd['response'][1])
#print(train_data_pd['context'][1][0]+train_data_pd['context'][1][1])
#print(len(train_data_pd['context'][3040]))

#test = train_data_pd['context'][10][0] + train_data_pd['context'][10][1] +train_data_pd['response'][10]
#print(test)
#test2 = train_data_pd['context'][:][0] + train_data_pd['context'][:][1] +train_data_pd['response']
all_stopwords = stopwords.words('english')
tk = SpaceTokenizer() 
ps = PorterStemmer()

for i in range(len(train_data_pd)):
    #train_data_pd['response'][i]=train_data_pd['response'][i]+train_data_pd['context'][i][0] + train_data_pd['context'][i][1]
    train_data_pd['response'][i]=train_data_pd['response'][i]+train_data_pd['context'][i][1]
    train_data_pd['response'][i]=train_data_pd['response'][i].replace('@USER', '').strip().lower()    
    text_tokens = tk.tokenize(train_data_pd['response'][i])
    tokens_without_sw = [word for word in text_tokens if not word in all_stopwords]
    test4=""
    for i in tokens_without_sw:
        test4 = test4 + " "+ps.stem(i)
    test4.strip()
    train_data_pd['response'][i]=test4
 
    


for i in range(len(test_data_pd)):
    #test_data_pd['response'][i]=test_data_pd['response'][i]+test_data_pd['context'][i][0] + test_data_pd['context'][i][1]
    test_data_pd['response'][i]=test_data_pd['response'][i]+test_data_pd['context'][i][1]
    test_data_pd['response'][i]=test_data_pd['response'][i].replace('@USER', '').strip().lower()

    text_tokens = tk.tokenize(test_data_pd['response'][i])
    tokens_without_sw = [word for word in text_tokens if not word in all_stopwords]
    test4=""
    for i in tokens_without_sw:
        test4 = test4 + " "+ps.stem(i)
    test4.strip()
    test_data_pd['response'][i]=test4
    
    
    
#print(train_data_pd['response'][1])
#print(len(train_data_pd['response'][1]))
print("Converted response PD data to include Context Data")
print("Converted response to lowercase and removed stop words as well as @USER")
print("Converted response to stem words using PortStemmer")


Converted response PD data to include Context Data
Converted response to lowercase and removed stop words as well as @USER
Converted response to stem words using PortStemmer


In [37]:
print(test_data_pd['response'][10])

define this way : 1 . desiring the good of the other ; wanting them to thrive / flourish , which means they'd get free from the attitudes you mention ; 2 . doing whatever's in your control / power to advance their good ; at least * not * wishing them ill , * not * hating them .  ok , you ’ re right , but how do you love someone who hates you , and wants you to not exist ? how do you love someone doesn ’ t share basic morals ?


In [38]:
test=train_data_pd['response'][0]+train_data_pd['context'][0][0] + train_data_pd['context'][0][1]
#print(test)
test2 = test.replace('@USER', '').strip().lower()
print(test2)
test3 = test2

all_stopwords = stopwords.words('english')
#text_tokens = word_tokenize(test3)
tk = SpaceTokenizer() 

text_tokens = tk.tokenize(test3)
tokens_without_sw = [word for word in text_tokens if not word in all_stopwords]
test4=""
for i in tokens_without_sw:
    test4 = test4 + " "+i
test4.strip()



#for j in stopwords.words('english'):
#    print(j)
#    test3=test3.replace(j, '')
#print(test3)


i don't get this .. obviously you do care or you would've moved right along .. instead you decided to care and troll her .. if your child isn't named barron ... #bebest melania couldn't care less . fact . 💯a minor child deserves privacy and should be kept out of politics . pamela karlan , you should be ashamed of your very angry and obviously biased public pandering , and using a child to do it . if your child isn't named barron ... #bebest melania couldn't care less . fact . 💯


"get .. obviously care would've moved right along .. instead decided care troll .. child named barron ... #bebest melania care less . fact . 💯a minor child deserves privacy kept politics . pamela karlan , ashamed angry obviously biased public pandering , using child . child named barron ... #bebest melania care less . fact . 💯"

In [39]:
#Define the vector of actual results:
Actual_Results = []    
for  l in data_train:
    if l['label'] == 'SARCASM':
        Actual_Results.append(1)
    else:
        Actual_Results.append(0)

In [40]:
## Import the various SKLearn ML models for testing:

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier 

In [41]:
# getting training dataset features and labels
features = train_data_pd['response']
labels = train_data_pd['label']
labels = Actual_Results


In [42]:

# Splitting of training data into train and test data
rawdata_train, rawdata_test, rawlabels_train, rawlabels_test = train_test_split(features, labels, test_size = .1, random_state = 0)

print("Training dataset split into this many train samples:")
print(len(rawdata_train))
print("Training dataset split into this many evvalidation samples:")
print(len(rawdata_test))





Training dataset split into this many train samples:
4500
Training dataset split into this many evvalidation samples:
500


In [43]:
train_df = pd.DataFrame({
    'text': rawdata_train.str.replace('@USER', '', regex=False).str.strip(),
    'labels': rawlabels_train
})
#rawdata_train, rawdata_test, rawlabels_train, rawlabels_test

In [44]:
eval_df = pd.DataFrame({
    'text': rawdata_test.str.replace('@USER', '', regex=False).str.strip(),
    'labels': rawlabels_test
})

In [45]:
train_df

,text,labels
3871,is the gig sold out ? no i'm not ... sorry . m...,0
4299,you will wait for the next hour - losing patie...,0
4719,that ’ s how clueless you are . the secdef was...,0
3195,i saw a demo . great job for taking the extra ...,0
1922,"yes , despite having lived in lincoln for many...",1
...,...,...
4931,you are condescendingly disrespectful and all ...,0
3264,"y'all need to be able to see my likes , so you...",0
1653,yep ! i've been asked ' will they add reviews ...,1
2607,my phone is a nokia 8 so it may be an upgrade ...,0


In [46]:
eval_df

,text,labels
398,countered with #climatechange activists stuck ...,1
3833,"when we share love , we get love too . our vib...",0
4836,give me nothing . just saying no one really kn...,0
4572,🍗 sinning happens nearly everywhere . as a res...,0
636,i was going for the . pv method gives mmp the ...,1
...,...,...
1928,here are my two favorite quotes that reflect c...,1
4366,very idea ! please call my #office ( the # 1 #...,0
3550,"you are very welcomed , lamar aĝa and shardall...",0
2189,so says someone who is follower of a religion ...,1


# Training and output


In [47]:
# Create a ClassificationModel
# for training from scratch
import torch

cuda_available = torch.cuda.is_available()
print("Does system have CUDA support?")
print(cuda_available)

model = ClassificationModel('roberta', 'roberta-base', use_cuda=cuda_available) # You can set class weights by using the optional weight argument



Does system have CUDA support?
True


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

In [49]:
model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "model_args.lazy_loading" : True,
    "use_early_stopping" : True,
    "num_train_epochs" : 4,
    "learning_rate": 2.741032760877178e-05,
    #"num_train_epochs" :13,
    #"learning_rate" : 0.000020
}

# Train the model
model.train_model(train_df,args=model_args)

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


(2252, 0.3643004748751914)

## Gen prediction


In [50]:
# getting training dataset features and labels
features_test = test_data_pd['response']
id_final_test = test_data_pd['id']

predictions_test, raw_outputs_test = model.predict(features_test)


In [51]:
#Writing the RoBERTa Classifier predictions to the output file: RoBERTa_answers.txt
y_pred = predictions_test
f = open("/content/gdrive/MyDrive/TxtClassComp_MattzeePrivate/RoBERTa_colab_answer.txt", "w")
for i in range(len(id_final_test)):
    i_result = y_pred[i]
    pred_id = id_final_test[i]
    if i_result == 1:
        f.write(pred_id + ',' + "SARCASM" +"\n")
    else:
        f.write(pred_id + ',' + "NOT_SARCASM" +"\n")
f.close()
